In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import os
from cleantext import clean
from pandas import DataFrame

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
import random as rnd

In [3]:
arr = os.listdir('.')
print(arr)

['Label_train_2.csv', '.DS_Store', 'Creation_matrice_Creation.ipynb', 'FinalDBelon.csv', 'Data_train.csv', 'DataNONEL_clean.csv', 'Clean_Tweets_EM.ipynb', 'Creation_matrice_Detection.ipynb', 'README.md', 'DataBase_temporary_creation.csv', 'Tweets_Not_ElonMusk.csv', 'DataBase.csv', '.gitignore', 'Reseau_neurone.ipynb', 'Data_train_2.csv', 'DataElon_clean.csv', '.ipynb_checkpoints', 'Label_train.csv', 'Clean_Tweets_NONEM.ipynb', '.git', 'TweetsElonMusk.csv']


In [4]:
assert hasattr(tf, 'function')

In [5]:
Data_train = pd.read_csv("Data_train.csv")
Label_train = pd.read_csv("Label_train.csv")

In [6]:
Data_train_2 = pd.read_csv('Data_train_2.csv')
Label_train_2 = pd.read_csv("Label_train_2.csv")

In [7]:
def Construction_modele(Data_train, Label_train, affichage=1):
    #Construction couche de neurones
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(2, activation="softmax"))
    #Mise au bon format des donnes pour le modele
    Tweet_1 = Data_train.values
    Tweet_2 = Tweet_1.astype(dtype='float32')
    Label_1 = Label_train.values
    Label_2 = Label_1[:,0].astype(dtype='uint8')
    #Ajustement modele et choix de metrique
    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    #Entrainement du modele
    model.fit(Tweet_2, Label_2, epochs=18, verbose=affichage)
    return model

In [8]:
def Vectorisation(Data_train, tweet):   #prend un tweet en argument sous la forme d'une string et renvoi son vecteur sous forme d'une database dans l espace vectoriel de la matrice
    Vecteurs = Data_train.columns
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    Mot_tweet = tweet.split(" ")
    
    for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary[mot] = index + 1
                
    row.append(dictionnary)
    Data_out = DataFrame(row)
    
    if Data_out.loc[0].max() != 0:
        Data_out.loc[0] = Data_out.loc[0] / Data_out.loc[0].max()
    
    return Data_out
    
    
    

In [9]:
def normalisation(x):
    if x.max() != 0:
        x = x / x.max()
    return x

In [10]:
def Vectorisation_liste(Data_train, liste_tweet, func_norma = normalisation):   
    Vecteurs = Data_train.columns
    
    row = []
    
    dictionnary = dict()
    for mot in Vecteurs:
        dictionnary[mot] = 0
    
    for tweet in liste_tweet:
        
        Mot_tweet = tweet.split(" ")
        dictionnary2 = dictionnary.copy()
        for index, mot in enumerate(Mot_tweet):
            if mot in Vecteurs:
                dictionnary2[mot] = index + 1
                
        row.append(dictionnary2)
        
    Data_out = DataFrame(row)
    Data_out = Data_out.apply(normalisation, axis=1)
        
    return Data_out

La fonction detection prend en argument un tweet sous la forme d'un string, le reseau de neurone entraine et la base de donnee d entrainement utilise pour vectorizer le tweet. La fonction renvoi un tuple de la forme
\
\
("Elon Musk detecte a x%", 1)
\
("Indetermine", 2)
\
("Pas Elon Musk a x%", 0)
\
\
1 est renvoye ssi x >= 60%
\
0 est renvoye ssi x >= 60%
\
2 est renvoye ssi la probabilite que ce soit Elon Musk (ou non) soit comprise entre 40% et 60%

In [11]:
def detection(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        char = "Indetermine"
        return (char, 2)
    Prediction = model_neurone.predict(vecteur)
    if Prediction[0, 1] >= 0.6:
        char = "Elon Musk detecte a " + str(Prediction[0, 1]) + "%"
        return (char, 1)
    elif Prediction[0,1] >= 0.4:
        char = "Indetermine"
        return (char, 2)
    else :
        char = "Pas Elon Musk a " + str(Prediction[0, 0]) + "%"
        return (char, 0)

In [12]:
def detection_creation(tweet_sentence, model_neurone, Data_train):
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    vecteur = vecteur_phrase.values
    vecteur = vecteur.astype(dtype='float32')
    if vecteur.sum() == 0:
        return 0
    Prediction = model_neurone.predict(vecteur)
    return Prediction[0, 1]

In [13]:
#Detecte et rend une estimation du tweet propose et l ajoute a son entrainement
#Renvoi lestimation et le nouveau model de neurone entraine apres estimation avec le tweet donne

def detection_apprentissage(tweet_sentence, model_neurone, Data_train, Label_train, val=0):
    #estimation
    (char, val) = detection(tweet_sentence, model_neurone, Data_train)
    #re entrainement du reseau de neurones
    tweet_sentence = tweet_sentence.lower()
    tweet_sentence = clean(tweet_sentence, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    vecteur_phrase = Vectorisation(Data_train, tweet_sentence)
    Data_train = pd.concat([Data_train, vecteur_phrase], ignore_index=True)
    Data_train.reset_index(drop=True, inplace=True)
    label = DataFrame([{'label':val}])
    Label_train = pd.concat([Label_train, label], ignore_index=True)
    Label_train.reset_index(drop=True, inplace=True)
    model = Construction_modele(Data_train, Label_train, affichage=0)
    return ((char, val), model)

In [14]:
#Prend en argument une liste de tweet sous la forme ["tweet1", "tweet2"…] et les labels associes sous la forme dune liste [0,0…] et renvoi une liste [estimation, model, Data_train, Label_train] où estimation est une liste des estimation faite de la liste de tweets passe, model est le reseau de neurones reentraines sur la liste de tweet apres avoir estimer et Data_train Et Label_train sont une concatenation des datasets donne et ceux utilise a exante 
def detection_apprentissage_liste(tweet_liste, model_neurone, Data_train, Label_train, val_list):
    Estimat = []
    Data_adding = Vectorisation_liste(Data_train, tweet_liste)
    Data = Data_train.copy()
    
    Label_adding = pd.DataFrame(val_list, columns=['label'])
    Label = Label_train.copy()
    
    Data_add = Data_adding.values
    Data_add = Data_add.astype(dtype='float32')
    
    valued = model_neurone.predict(Data_add)
    
    Data = pd.concat([Data, Data_adding], ignore_index = True)
    Label = pd.concat([Label, Label_adding], ignore_index = True)
    
    model_neurone = Construction_modele(Data, Label, affichage=0)
    
    valeur = (valued[:,1] > 0.5).astype(dtype='int').tolist()

    
    return [model_neurone, Data, Label, tweet_liste, valeur]
    
    

In [15]:
tweet_list = ['test1', 'test2', 'space fake', 'fake news']

In [ ]:
[model, Data, Label, tweetl, val] = detection_apprentissage_liste(tweet_list, reseau_neurone_detection, Data_train, Label_train, [0,0,0,0])



In [ ]:
detection('i love my fan', model, Data)

In [ ]:
#Initialisation des reseaux de neurones avant le dialogue

In [16]:
reseau_neurone_detection = Construction_modele(Data_train, Label_train)

Epoch 1/18
348/348 [==============================] - 1s 1ms/step - loss: 0.6873 - accuracy: 0.5474
Epoch 2/18
348/348 [==============================] - 0s 1ms/step - loss: 0.6671 - accuracy: 0.6642
Epoch 3/18
348/348 [==============================] - 0s 1ms/step - loss: 0.6351 - accuracy: 0.7184
Epoch 4/18
348/348 [==============================] - 0s 1ms/step - loss: 0.5861 - accuracy: 0.7522
Epoch 5/18
348/348 [==============================] - 0s 1ms/step - loss: 0.5257 - accuracy: 0.7779
Epoch 6/18
348/348 [==============================] - 0s 1ms/step - loss: 0.4685 - accuracy: 0.8031
Epoch 7/18
348/348 [==============================] - 0s 1ms/step - loss: 0.4230 - accuracy: 0.8223
Epoch 8/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3888 - accuracy: 0.8376
Epoch 9/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3619 - accuracy: 0.8469
Epoch 10/18
348/348 [==============================] - 0s 1ms/step - loss: 0.3416 - accuracy: 0.8531

In [17]:
reseau_neurone_creation = Construction_modele(Data_train_2, Label_train_2)

Epoch 1/18
370/370 [==============================] - 1s 2ms/step - loss: 0.6897 - accuracy: 0.5377
Epoch 2/18
370/370 [==============================] - 0s 1ms/step - loss: 0.6817 - accuracy: 0.5887
Epoch 3/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6713 - accuracy: 0.6398
Epoch 4/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6553 - accuracy: 0.6750
Epoch 5/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6312 - accuracy: 0.7016
Epoch 6/18
370/370 [==============================] - 1s 1ms/step - loss: 0.6000 - accuracy: 0.7181
Epoch 7/18
370/370 [==============================] - 1s 1ms/step - loss: 0.5662 - accuracy: 0.7400
Epoch 8/18
370/370 [==============================] - 1s 1ms/step - loss: 0.5347 - accuracy: 0.7575
Epoch 9/18
370/370 [==============================] - 1s 1ms/step - loss: 0.5073 - accuracy: 0.7700
Epoch 10/18
370/370 [==============================] - 1s 1ms/step - loss: 0.4836 - accuracy: 0.7856

In [ ]:
#Debut du dialogue

In [18]:
def Creation_tweet(nombre_mots_total, phrase, Data_train_2, model_neurone): #renvoi un tweet sous la forme d une phrase 
    phrase = phrase.lower()
    phrase = clean(phrase, no_emoji=True, no_punct=True, no_urls = True, replace_with_url='', no_digits=True, replace_with_digit='', no_currency_symbols=True, replace_with_currency_symbol='')
    liste_mot =  phrase.split(" ")
    if nombre_mots_total < len(liste_mot):
        print("Erreur liste mot plus longue que le tweet prevu, longueur du tweet pris par defaut au nombre de mots")
        nombre_mots_total = len(liste_mot)
    nombre_mot_rest = nombre_mots_total - len(liste_mot)
    
    liste_tweet = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
    tweet_pertinent = ' '.join(liste_tweet)
    pertinence = detection_creation(tweet_pertinent, model_neurone, Data_train_2)
    
    for iteration in range(100):
        if pertinence > 0.9:
            tweet_pertinent = clean(tweet_pertinent, no_punct=True)
            return tweet_pertinent
        liste = liste_mot + rnd.sample(list(Data_train_2.columns), nombre_mot_rest, counts=None)
        rnd.shuffle(liste)
        current_tweet = ' '.join(liste)
        current_pertinence = detection_creation(current_tweet, model_neurone, Data_train_2)
        if current_pertinence > pertinence:
            pertinence = current_pertinence
            tweet_pertinent = current_tweet
    current = clean(current_tweet, no_punct=True)
    return current
        
    

In [27]:
tweetos = Creation_tweet(8, '', Data_train_2, reseau_neurone_creation)

In [28]:
tweetos

'very then needed start break trip plan'

In [ ]:
detection(tweetos, reseau_neurone_creation, Data_train_2)

In [19]:
Data_train_Detection = Data_train.copy()
Data_train_Creation = Data_train_2.copy()

Label_train_Detection = Label_train.copy()
Label_train_Creation = Label_train_2.copy()

In [20]:
def dialogue(iteration, nombre_mot ,reseau_neurone_detection, reseau_neurone_creation, Data_train, Data_train_2, Label_train, Label_train_2):

    for i in range(iteration):
        
        #Creation parle en premier et propose 30 tweets, qu il juge pertinent, a Detection
        print('Tweet propose par creation a la ', i , 'eme iteration : \n')
        tweet_liste = []
        valeur = [0]*30 
        for j in range(30):
            
            tweet_proposal = Creation_tweet(nombre_mot, '', Data_train_2, reseau_neurone_creation)
            print(tweet_proposal)
            tweet_liste.append(tweet_proposal)
        print('\n')
        
        #Detection recoit ces 30 tweets, les evalues, les appreds en tenant compte qu ils sont faux, et retourne a creation ce qu il en pensait avant d apprendre qu ils etaient faux
        
        [reseau_neurone_detection, Data_train, Label_train, tweet_liste, evaluation] = detection_apprentissage_liste(tweet_liste, reseau_neurone_detection, Data_train, Label_train, valeur)
        
        #Affichage de ce que globalement detection en pensait
        
        print('Sur ces tweets propose detection en a trouve :', sum(evaluation), ' correct selon lui')
        
        #Creation apprend de ses tweets en les labelisant suivant ce que detection en avait pense a priori
        
        [reseau_neurone_creation, Data_train_2, Label_train_2, _, _] = detection_apprentissage_liste(tweet_liste, reseau_neurone_creation, Data_train_2, Label_train_2, evaluation)
        
        print('\n \n')
        
        
            
            
            
            
            

In [21]:
dialogue(1, 8, reseau_neurone_detection, reseau_neurone_creation, Data_train_Detection, Data_train_Creation, Label_train_Detection, Label_train_Creation)

Tweet propose par creation a la  0 eme iteration : 

shot where lets company tweets fair definitely
any park ~ especially monday die problem
slow off few didnt that listening climate
please why cover yes let california problem
best date side vevo weve sun software
tell taking post tough exactly battery tomorrow
rolling review build works close supercharger prob
final collection asia production risk per build
hw already fashion god vegas yes covered
a water pass because autopilot body sign
need aint pressure night california once spacex
tell minimum shop yeah americans asked engineering
lol if quality stream after launch idea
us orbit call tunnel old software cause
totally list advanced making rear best ai
same during launch month obamacare may story
moon seem question found asked tesla takes
part fair unless in address obamacare lower
lucky hard think test production given london
friday velocity by america teslamotors earth isnt
giant london minimum reality covered + force
wouldnt thin

In [ ]:
phrase = 'I sing every morning in my shower, i love space and tesla'
#reponse = detection(phrase, reseau_neurone_creation, Data_train_2)
reponse = detection_apprentissage(phrase, reseau_neurone_detection, Data_train, Label_train)


In [ ]:
((_, val), _) = reponse
val